In [1]:
%env JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
%env PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin

env: JAVA_HOME=/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64
env: PATH=/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/bin:storage/icds/swst/deployed/production/20220813/apps/anaconda3/2021.05_gcc-8.5.0/bin:/usr/lib/jvm/java-11-openjdk-11.0.25.0.9-2.el8.x86_64/bin/java:/usr/local/bin:/usr/bin:/usr/local/sbin:/usr/sbin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin:/storage/icds/tools/bin:/storage/sys/slurm/bin


In [1]:
import findspark
import pandas as pd
import numpy as np

import pyspark as psk
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql import functions as f
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, StringType
from pyspark.sql import DataFrame
from pyspark.sql.functions import *

import time as t 
from datetime import date, datetime, timedelta
import joblib
from joblib import Parallel, delayed

import multiprocessing
from functools import reduce  
from math import modf
import sqlite3
import os
import sys

from delta import * 
from delta.tables import *
from delta import configure_spark_with_delta_pip
# import covalent as ct

import subprocess
import re
import random

from sklearn.preprocessing import OneHotEncoder
from sklearn.decomposition import PCA
# import umap.umap_ as umap
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import matplotlib.pyplot as plt

import gc  # import gorbage collector to resolve the problem of restarting kernel due to large table of population loading in RAM to append
from pyspark.sql.functions import year, col

findspark.init()
findspark.find()

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

# os.environ['PYDEVD_DISABLE_FILE_VALIDATION']=1

spark = SparkSession.builder \
       .config("spark.ui.port", "4050") \
       .master("local[2]") \
       .appName("MyApp") \
       .config("spark.driver.maxResultSize", "40g")\
       .config("spark.driver.memory", "140g")\
       .getOrCreate()
sc = spark.sparkContext

# sqlContext = SQLContext(sc)


#both works
# 1: 
# spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")    #To resolve the error for p1075_38 to_timestamp formating: You may get a different result due to the upgrading to Spark >= 3.0: Fail to parse '1/2/2019 20:40:00' in the new parser. You can set spark.sql.legacy.timeParserPolicy to LEGACY to restore the behavior before Spark 3.0, or set to CORRECTED and treat it as an invalid datetime string.
# Set Spark SQL legacy time parser policy to LEGACY to handle older date formats
# 2:
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")
# Increase the max fields in the string representation of a plan
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
spark.conf.set("spark.sql.debug.maxToStringFields", 2000)

ps: /storage/icds/RISE/sw8/anaconda/anaconda3/envs/tensorflow/lib/libuuid.so.1: no version information available (required by /usr/lib64/libblkid.so.1)
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/22 23:24:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/data/dataset/VINs_data(problem).csv") \
          .createOrReplaceTempView("VINs_data")

spark.read.option("header",True) \
          .csv("/storage/home/yqf5148/work/volvoPennState/PopulationWithChassisId.csv") \
          .createOrReplaceTempView("population")
df_population = spark.sql("SELECT * FROM population")

In [18]:
print(df_population.count())

322178


In [3]:
# Get all column names from the population table
all_columns = spark.table("population").columns

# Select only columns that contain "KOLA" and exclude "CHASSIS_ID"
kola_columns = [c for c in all_columns if "KOLA" in c and c != "CHASSIS_ID"]

# Build the main DataFrame with selected columns
df_population = spark.table("population") \
    .select(
        col("VIN"),                               # Include VIN
        year("VEH_ASSEMB_DATE").alias("VEH_ASSEMB_YEAR"),  # Extract year from VEH_ASSEMB_DATE
        *[col(c) for c in kola_columns]           # Include all KOLA-related columns
    )

df_population.show()


+-----------------+---------------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+-----

In [7]:
kola_cols = [c for c in df_population.columns if "KOLA" in c]
non_kola_cols = [c for c in df_population.columns if c not in kola_cols]

df_non_kola = df_population[non_kola_cols]
df_kola = df_population[kola_cols]
df_kola_pd = df_kola.toPandas()  # convert from PySpark to Pandas

ERROR:root:Exception while sending command.                                     
Traceback (most recent call last):
  File "/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


AssertionError: Undefined error message parameter for error class: CANNOT_PARSE_DATATYPE. Parameters: {'error': 'An error occurred while calling o1837.schema'}

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/storage/home/yqf5148/work/anaconda3/envs/volvopennstate-env/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [6]:
df_kola = df_kola.astype('category')

AttributeError: 'DataFrame' object has no attribute 'astype'

In [23]:
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
kola_encoded = encoder.fit_transform(df_kola)

print("Original shape:", df_kola.shape)
print("Encoded shape:", kola_encoded.shape)

ValueError: Expected 2D array, got scalar array instead:
array=DataFrame[_KOLA_01X: string, _KOLA_02X: string, _KOLA_03X: string, _KOLA_04X: string, _KOLA_05X: string, _KOLA_06X: string, _KOLA_07X: string, _KOLA_08A: string, _KOLA_08X: string, _KOLA_09X: string, _KOLA_0AX: string, _KOLA_0BX: string, _KOLA_0CX: string, _KOLA_0DX: string, _KOLA_0EA: string, _KOLA_0EX: string, _KOLA_0GX: string, _KOLA_0HA: string, _KOLA_0HX: string, _KOLA_0JX: string, _KOLA_0KX: string, _KOLA_0LX: string, _KOLA_0NX: string, _KOLA_0PA: string, _KOLA_0PX: string, _KOLA_0RA: string, _KOLA_0RX: string, _KOLA_0SX: string, _KOLA_15X: string, _KOLA_19A: string, _KOLA_1BX: string, _KOLA_1CX: string, _KOLA_1DA: string, _KOLA_1DX: string, _KOLA_1EA: string, _KOLA_1GA: string, _KOLA_1HA: string, _KOLA_1JX: string, _KOLA_1KX: string, _KOLA_1LA: string, _KOLA_1NX: string, _KOLA_1PB: string, _KOLA_1PX: string, _KOLA_1QB: string, _KOLA_1RB: string, _KOLA_1RX: string, _KOLA_1SB: string, _KOLA_1SX: string, _KOLA_1TX: stri